In [ ]:
import json
from openai import OpenAI
import pandas as pd

In [ ]:
client = OpenAI()

In [ ]:
messages = [{"role": "user", "content": "Hola, como estás?"}]
response = client.chat.completions.create(model="gpt-4.1-nano", messages=messages)

In [ ]:
response.choices[0].message.content

In [ ]:
def get_store_locations(postal_codes: list[int]) -> list:
    """
    Retrieves the locations of stores near the specified postal codes.

    Args:
        postal_codes (list): A list of postal codes to search for nearby stores.

    Returns:
        list: A list of store locations found near the provided postal codes.
    """
    print(f"Function called for postal codes: {postal_codes}")
    df = pd.read_csv("../data/stores.csv")
    filtered_df = df[df["postal_code"].isin(postal_codes)]
    return list(filtered_df["store_name"].values)

In [ ]:
get_store_locations([27003, 15001, 15003, 27004])

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_store_locations",
            "description": "Retrieve the names of stores near the specified postal codes.",
            "parameters": {
                "type": "object",
                "properties": {
                    "postal_codes": {
                        "type": "array",
                        "items": {"type": "integer"},
                        "description": "List of postal codes to search for nearby stores. Example: [15003, 15704]",
                    }
                },
                "required": ["postal_codes"],
                "additionalProperties": False,
            },
        },
    }
]

In [ ]:
globals().get("get_store_locations")([27003, 15001, 15003, 27004])

In [ ]:
def handle_tool_calls(tool_calls: list) -> list:
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)

        tool = globals().get(tool_name)
        result = tool(**args) if tool else {}

        results.append(
            {
                "role": "tool",
                "content": json.dumps(result),
                "tool_call_id": tool_call.id,
            }
        )
    return results

In [ ]:
from queue import Queue, Empty
from contextlib import contextmanager
from chromadb import PersistentClient

class ChromaDbClient:

    def __init__(self, db_path: str = './chroma', pool_size: int = 10):
        self.pool = Queue(maxsize=pool_size)
        succesful_connections = 0
        for i in range(pool_size):
            try:
                client = PersistentClient(db_path)
                self.pool.put(client)
                succesful_connections += 1
            except Exception as e:
                print(f'Error while creating connection {i+1}: {str(e)}')
        
        if self.pool.empty():
            raise RuntimeError('Unable to establish connections with the database')
        
        print(f'{succesful_connections}/{pool_size} connections established')
    
    def close(self):
        while not self.pool.empty():
            self.pool.get_nowait()

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.close()

    @contextmanager
    def acquire(self, collection_name: str, timeout: float = 30):
        try:
            client = self.pool.get(timeout=timeout)
        except Empty:
            raise RuntimeError('No available connections in pool')
        try:
            yield client.get_or_create_collection(collection_name)
        finally:
            self.pool.put(client)

In [ ]:
chroma_client = ChromaDbClient('../chroma', pool_size=5)

In [ ]:
def get_product_info(question: str) -> list[str]:
    """
    Retrieves the most relevant text chunks based on semantic similarity from a ChromaDB collection.

    This function uses a vector similarity search to find and return the text fragments 
    most relevant to a product-related question.

    Args:
        question (str): A natural language question about one or more products.

    Returns:
        list[str]: A list of text chunks that are most similar to the input question.
    """
    print(f'Function called with query: {question}')
    try:

        with chroma_client.acquire('products') as collection:
            results = collection.query(query_texts=[question], n_results=5)

        documents = results.get('documents', [])
        if not documents:
            return [f'No relevant documents found for the query: {question}']

        return documents
    except Exception as e:
        print(e)
        return [f'An error occurred while retrieving related documents']

In [ ]:
globals().get('get_product_info')('Qué alérgenos tienen las patatas?')

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_store_locations",
            "description": "Retrieve the names of stores near the specified postal codes.",
            "parameters": {
                "type": "object",
                "properties": {
                    "postal_codes": {
                        "type": "array",
                        "items": {"type": "integer"},
                        "description": "List of postal codes to search for nearby stores. Example: [15003, 15704]",
                    }
                },
                "required": ["postal_codes"],
                "additionalProperties": False,
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_product_info",
            "description": "Retrieve relevant product information based on a natural language question using semantic similarity search",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "A natural language question about one or more products. Example: 'What are the allergens in chocolate?'",
                    }
                },
                "required": ["question"],
                "additionalProperties": False,
            },
        },
    }
]

In [ ]:
system_prompt = """
Eres un asistente de atención al cliente encargado de ayudar a los usuarios a localizar tiendas cercanas y proporcionar información sobre productos.

Dispones de dos herramientas:

1. `get_store_locations`: permite recuperar las tiendas disponibles cerca de una ubicación.  
   Esta herramienta recibe un único array con varios códigos postales, por ejemplo: [28001, 28002, 28003], y devuelve las tiendas que hay en esas zonas.

2. `get_product_info`: permite recuperar información relevante sobre productos basada en preguntas en lenguaje natural.

INSTRUCCIONES:

1. Si el usuario menciona una ciudad:
   - Identifica el nombre exacto de la ciudad.
   - Obtén todos los códigos postales que pertenecen a esa ciudad.
     No uses solo un código o unos pocos: asegúrate de generar la lista completa de códigos postales asociados a esa ciudad.
   - Llama a la herramienta `get_store_locations` pasando un único array con todos esos códigos postales.

2. Si el usuario menciona una ciudad y un distrito o barrio específico, asegúrate igualmente de incluir todos los códigos postales que correspondan a ese ámbito, aunque sean solo un subconjunto de la ciudad.

3. Si el usuario formula una pregunta relacionada con productos (ingredientes, alérgenos, características, etc.), utiliza la herramienta `get_product_info` para obtener la información relevante, pasando la pregunta completa como parámetro.

4. Si el usuario pregunta sobre cómo ponerse en contacto con la empresa, responde con el siguiente texto:
   "Puedes llamarnos al teléfono 888 888 888 o escribir un correo electrónico a test@test.com."

5. No agrupes llamadas por código individual ni repitas llamadas con arrays parciales.

6. Si el usuario pregunta qué tipo de consultas puedes responder, explica que puedes ayudar a localizar tiendas cercanas por ciudad o código postal, y proporcionar información sobre productos.

7. Si el usuario pregunta algo que no está relacionado con localizar tiendas, información sobre productos, ni cómo ponerse en contacto, responde:
   "Lo siento, no puedo ayudarte con esa consulta."
"""

In [ ]:
system_message = [{"role": "system", "content": system_prompt}]

In [ ]:
def chat(message, history):
    messages = system_message + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=messages,
            tools=tools,
            parallel_tool_calls=False,
        )
        if response.choices[0].finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
import gradio as gr

gr.ChatInterface(chat, type="messages").launch(share=False)